In [1]:
import cv2
import numpy as np
from datetime import datetime
import pandas as pd
from pathlib import Path
import os

# Trafik renk kodları
color_to_code = {
    (22,224,152): 1,
    (255,207,67): 2,
    (242,78,66): 3,
    (169,39,39): 4,
    (216,224,231): 0  # Gri - trafik yok
}


def get_closest_color_code(rgb_pixel):
    min_dist = float('inf')
    closest_code = 0
    for ref_rgb, code in color_to_code.items():
        dist = np.linalg.norm(np.array(rgb_pixel) - np.array(ref_rgb))
        if dist < min_dist:
            min_dist = dist
            closest_code = code
    return closest_code

def analyze_image(image_path, kavsak_adi, yol_konumlari, csv_path="trafik_kayit_final.csv"):
    image_path = Path(image_path)
    image = cv2.imread(str(image_path))

    if image is None:
        print(f"❌ Görsel yüklenemedi: {image_path}")
        return

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Zamanı dosya adından al
    filename = image_path.stem
    try:
        parts = filename.split("_")
        if len(parts) >= 3:
            gun = int(parts[1])
            saat = int(parts[2][:2])
            dakika = int(parts[2][2:4])
            saniye = int(parts[2][4:6])
            zaman = f"2025-04-{gun:02d} {saat:02d}:{dakika:02d}:{saniye:02d}"
        else:
            raise ValueError("Dosya adı formatı hatalı")
    except Exception as e:
        print(f"⚠️ Zaman ayrıştırılamadı, sistem zamanı kullanılacak: {e}")
        zaman = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    kayıtlar = []
    for yol_ismi, noktalar in yol_konumlari.items():
        renk_kodlari = []
        for (x, y) in noktalar:
            pixel = image_rgb[y, x]
            kod = get_closest_color_code(pixel)
            renk_kodlari.append(kod)
        kayıtlar.append([zaman, kavsak_adi, yol_ismi, *renk_kodlari])

    df = pd.DataFrame(kayıtlar, columns=["zaman", "kavsak_adi", "yol_ismi", "bas_rengi", "orta_rengi", "son_rengi"])

    try:
        df.to_csv(csv_path, mode='a', header=not Path(csv_path).exists(), index=False)
        print(f"✅ CSV'ye kayıt eklendi: {image_path.name} → {zaman} ({kavsak_adi})")
    except Exception as e:
        print("CSV yazma hatası:", e)

# Her kavşağın yol koordinatları
kavsaklar = {
    "Kavsak 1": {
        "Hatboyu-2 Cd.": [(1564, 345), (1630, 330), (1698, 315)],
        "Gazi Yakup Satar Cd. (Kuzey)": [(1547, 340), (1565, 307), (1609, 225)],
        "Gazi Yakup Satar Cd. (Guney)": [(1544, 384), (1533, 416), (1524, 440)],
        "Zubeyde Hanim Blv. (Sag)": [(1515, 387), (1473, 397), (1431, 407)],
    },
    "Kavsak 2": {
        "Sakarya-2 Cd.": [(1094, 443), (1094, 409), (1098, 359)],
        "Zubeyde Hanim Blv. (Sag)": [(1122, 447), (1153, 440), (1251, 418)],
        "Zubeyde Hanim Blv. (Sol)": [(1043, 492), (961, 510), (926, 517)],
        "Sakarya-1 Cd.": [(1109, 485), (1122, 536), (1131, 569)]
    },
    "Kavsak 3": {
        "Cengiz Topel Cd.": [(860, 532), (893, 573), (942, 627)],
        "Zubeyde Hanim Blv. (Sol)": [(855, 507), (882, 502), (935, 487)],
        "Yilmaz Buyukersen Blv.": [(826, 509), (810, 490), (783, 462)]
    }
}


# Görsellerin bulunduğu klasör
image_folder = "/kaggle/input/kavsakdata/SS2/SS2"

# Tüm resimler için
for filename in sorted(os.listdir(image_folder)):
    image_path = os.path.join(image_folder, filename)
    for kavsak_adi, yol_konumlari in kavsaklar.items():
        analyze_image(image_path, kavsak_adi, yol_konumlari, csv_path="trafik_kayit_final.csv")


✅ CSV'ye kayıt eklendi: screenshot_06_202102.png → 2025-04-06 20:21:02 (Kavsak 1)
✅ CSV'ye kayıt eklendi: screenshot_06_202102.png → 2025-04-06 20:21:02 (Kavsak 2)
✅ CSV'ye kayıt eklendi: screenshot_06_202102.png → 2025-04-06 20:21:02 (Kavsak 3)
✅ CSV'ye kayıt eklendi: screenshot_06_202113.png → 2025-04-06 20:21:13 (Kavsak 1)
✅ CSV'ye kayıt eklendi: screenshot_06_202113.png → 2025-04-06 20:21:13 (Kavsak 2)
✅ CSV'ye kayıt eklendi: screenshot_06_202113.png → 2025-04-06 20:21:13 (Kavsak 3)
✅ CSV'ye kayıt eklendi: screenshot_06_202124.png → 2025-04-06 20:21:24 (Kavsak 1)
✅ CSV'ye kayıt eklendi: screenshot_06_202124.png → 2025-04-06 20:21:24 (Kavsak 2)
✅ CSV'ye kayıt eklendi: screenshot_06_202124.png → 2025-04-06 20:21:24 (Kavsak 3)
✅ CSV'ye kayıt eklendi: screenshot_06_202134.png → 2025-04-06 20:21:34 (Kavsak 1)
✅ CSV'ye kayıt eklendi: screenshot_06_202134.png → 2025-04-06 20:21:34 (Kavsak 2)
✅ CSV'ye kayıt eklendi: screenshot_06_202134.png → 2025-04-06 20:21:34 (Kavsak 3)
✅ CSV'ye kayıt e